<a href="https://colab.research.google.com/github/7981888365/Retail-Sales-Rule-Based-Expert-System/blob/main/Retail_sales_DSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction**

This project develops a Rule-Based Retail Sales Expert System that evaluates sales patterns and detects potential business risks using IF–THEN decision rules. By analysing monthly sales trends, category-wise performance, customer concentration, pricing behaviour, and revenue fluctuations, the system classifies the overall business condition into risk levels such as Low, Moderate, High, or Critical.

The expert system offers clear explanations for each decision, making it transparent, easy to understand, and suitable for academic learning. It demonstrates how rule-based reasoning can convert raw sales data into actionable insights, helping retailers recognize declining trends, unstable revenue patterns, or overdependence on specific customers. This project highlights the practical value of expert systems in improving decision-making and ensuring better business planning


**Why This Project?**

Retail businesses depend heavily on understanding sales trends and risks.

Retail decisions naturally suit IF–THEN rule-based logic, making it perfect for an expert system.

The system is fully explainable, easy to understand, and ideal for academic evaluation.

It provides practical insights such as identifying declining categories, unstable sales periods, and overreliance on specific customers.

A safe, real-world, and highly relevant domain for demonstrating expert system concepts.

In [ ]:
"""
Retail Risk Screener - Rule-Based Expert System
Adapted from a mental-health IF-THEN pattern to detect retail business risks
against /content/retail_sales_dataset.csv

Outputs:
    risk_label (str), explanations (list of str), diagnostics (dict of computed metrics)
"""

import pandas as pd

def load_data(path="/content/retail_sales_dataset.csv"):
    df = pd.read_csv(path)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

def compute_metrics(df):
    metrics = {}

    # overall totals
    metrics["total_revenue"] = df["Total Amount"].sum()
    metrics["total_transactions"] = len(df)
    metrics["avg_order_value"] = df["Total Amount"].mean()
    metrics["avg_quantity"] = df["Quantity"].mean()

    # monthly sales timeseries (sorted by month)
    monthly = df.groupby(df["Date"].dt.to_period("M"))["Total Amount"].sum().sort_index()
    monthly.index = monthly.index.to_timestamp()
    metrics["monthly_sales_series"] = monthly

    # category sales and share
    cat_sales = df.groupby("Product Category")["Total Amount"].sum().sort_values(ascending=False)
    metrics["category_sales"] = cat_sales
    metrics["category_share"] = (cat_sales / metrics["total_revenue"]).to_dict()

    # last 3 months and previous 3 months sums (if available)
    m = monthly
    if len(m) >= 6:
        metrics["last_3m_sum"] = m[-3:].sum()
        metrics["prev_3m_sum"] = m[-6:-3].sum()
    else:
        metrics["last_3m_sum"] = m[-3:].sum() if len(m) >= 3 else m.sum()
        metrics["prev_3m_sum"] = m[:-3].sum() if len(m) > 3 else 0.0

    # month over month change for the last month (if available)
    if len(m) >= 2:
        metrics["last_mom_pct"] = (m.iloc[-1] - m.iloc[-2]) / (m.iloc[-2] if m.iloc[-2] != 0 else 1) * 100
    else:
        metrics["last_mom_pct"] = 0.0

    # per-category recent trend (last 3 months)
    recent_period = df[df["Date"] >= (df["Date"].max() - pd.DateOffset(months=3))]
    cat_recent = recent_period.groupby("Product Category")["Total Amount"].sum()
    metrics["category_recent_sales"] = cat_recent.to_dict()

    # top-customer concentration: share of revenue from top 1 and top 10 customers
    cust_rev = df.groupby("Customer ID")["Total Amount"].sum().sort_values(ascending=False)
    metrics["top1_customer_share"] = cust_rev.iloc[0] / metrics["total_revenue"] if len(cust_rev) > 0 else 0.0
    metrics["top10_customer_share"] = cust_rev.iloc[:10].sum() / metrics["total_revenue"] if len(cust_rev) >= 1 else 0.0

    # pricing vs quantity by category (avg price per unit, avg qty)
    cat_price = df.groupby("Product Category")["Price per Unit"].mean()
    cat_qty = df.groupby("Product Category")["Quantity"].mean()
    metrics["category_avg_price"] = cat_price.to_dict()
    metrics["category_avg_quantity"] = cat_qty.to_dict()

    # volatility measure: standard deviation of monthly sales relative to mean
    metrics["monthly_sales_volatility_pct"] = (m.std() / (m.mean() if m.mean() != 0 else 1)) * 100 if len(m) > 1 else 0.0

    return metrics

def retail_risk_assessment(df, focus_category=None):
    """
    df: pandas DataFrame of the retail dataset
    focus_category: optional string. If provided, apply rules specifically to that category.
    Returns: (risk_label, explanations, diagnostics)
    """

    explanations = []
    metrics = compute_metrics(df)

    # helper shortcuts
    last_mom = metrics["last_mom_pct"]
    last_3m = metrics["last_3m_sum"]
    prev_3m = metrics["prev_3m_sum"]
    top1_share = metrics["top1_customer_share"]
    top10_share = metrics["top10_customer_share"]
    vol_pct = metrics["monthly_sales_volatility_pct"]

    # If user asked for a particular category, compute category-specific numbers
    if focus_category:
        cat = focus_category
        cat_total = metrics["category_sales"].get(cat, 0.0)
        cat_recent = metrics["category_recent_sales"].get(cat, 0.0)
        cat_avg_price = metrics["category_avg_price"].get(cat, None)
        cat_avg_qty = metrics["category_avg_quantity"].get(cat, None)
    else:
        cat = None
        cat_total = None
        cat_recent = None
        cat_avg_price = None
        cat_avg_qty = None

    # -------------------------
    # CRITICAL RISK rules (immediate attention)
    # -------------------------
    # Rule CRIT-1: Month-over-month collapse (last month sales decreased > 70%)
    if last_mom < -70:
        explanations.append("Rule CRIT-1: Last month's sales dropped by more than 70% (large sudden collapse). Immediate investigation required.")
        return "CRITICAL RISK – Immediate attention required", explanations, metrics

    # Rule CRIT-2: Extreme customer concentration (top 1 customer > 70% revenue)
    if top1_share >= 0.70:
        explanations.append(f"Rule CRIT-2: Top customer contributes {top1_share:.0%} of revenue → Critical concentration risk (single customer failure would cripple revenue).")
        return "CRITICAL RISK – Immediate attention required", explanations, metrics

    # Rule CRIT-3: A previously high-share category lost almost all sales last period
    if focus_category and cat_total is not None:
        # if category had >40% share historically but recent 3-month is near zero
        historic_share = metrics["category_share"].get(cat, 0.0)
        if historic_share >= 0.40 and cat_recent < 1e-3:
            explanations.append(f"Rule CRIT-3: Category '{cat}' had {historic_share:.0%} historical share but shows near-zero recent sales.")
            return "CRITICAL RISK – Immediate attention required", explanations, metrics

    # -------------------------
    # HIGH RISK rules (action soon)
    # -------------------------
    # Rule HIG-1: Last 3 months declined > 30% vs previous 3 months
    if prev_3m > 0 and (prev_3m - last_3m) / prev_3m >= 0.30:
        drop_pct = (prev_3m - last_3m) / prev_3m * 100
        explanations.append(f"Rule HIG-1: Recent 3-month revenue down {drop_pct:.1f}% vs previous 3 months → High risk of downward trend.")
        return "HIGH RISK – Investigate & act soon", explanations, metrics

    # Rule HIG-2: Top-10 customers > 60% of revenue (over-reliance)
    if top10_share >= 0.60:
        explanations.append(f"Rule HIG-2: Top 10 customers account for {top10_share:.0%} of revenue → Customer concentration risk.")
        return "HIGH RISK – Investigate & diversify customer base", explanations, metrics

    # Rule HIG-3: High volatility in monthly sales (>50% std/mean)
    if vol_pct >= 50:
        explanations.append(f"Rule HIG-3: Monthly sales volatility is high ({vol_pct:.0f}% of mean) → Risk of unpredictable cashflow.")
        return "HIGH RISK – Stabilize inventory/marketing", explanations, metrics

    # Rule HIG-4: Price increase accompanied by falling quantities in a category (possible pricing issue)
    if focus_category and cat_avg_price is not None and cat_avg_qty is not None:
        # simple heuristic: if avg price > overall avg_price * 1.5 and avg quantity < overall avg_quantity * 0.8
        overall_avg_price = df["Price per Unit"].mean()
        overall_avg_qty = df["Quantity"].mean()
        if (cat_avg_price > 1.5 * overall_avg_price) and (cat_avg_qty < 0.8 * overall_avg_qty):
            explanations.append(f"Rule HIG-4: Category '{cat}' priced higher than average but sells lower quantity → possible pricing/sensitivity issue.")
            return "HIGH RISK – Reevaluate pricing for category", explanations, metrics

    # -------------------------
    # MODERATE RISK rules (monitor & prepare)
    # -------------------------
    # Rule MOD-1: Mild downward momentum: last month down 10-30%
    if -30 <= last_mom < -10:
        explanations.append(f"Rule MOD-1: Last month down {last_mom:.1f}% vs prior month → Monitor and consider targeted promotions.")
        return "MODERATE RISK – Monitor and prepare actions", explanations, metrics

    # Rule MOD-2: Category share decreased materially (>20%) compared to overall (e.g., competitor impact)
    if focus_category and cat_total is not None:
        historic_share = metrics["category_share"].get(cat, 0.0)
        # compare that category recent / historic (if historic non-zero)
        recent_share = cat_recent / (metrics["total_revenue"] if metrics["total_revenue"] != 0 else 1)
        if historic_share > 0 and (historic_share - recent_share) / historic_share >= 0.20:
            explanations.append(f"Rule MOD-2: Category '{cat}' share fell by at least 20% vs historical share → Potential market-share loss.")
            return "MODERATE RISK – Investigate category performance", explanations, metrics

    # Rule MOD-3: Low avg order value combined with low quantity (possible pricing or product-mix issues)
    if metrics["avg_order_value"] < df["Total Amount"].median() * 0.8 and metrics["avg_quantity"] <= 2:
        explanations.append("Rule MOD-3: Low average order value and low quantities → Consider bundles, upsells, or promotions.")
        return "MODERATE RISK – Consider conversion and assortment tactics", explanations, metrics

    # -------------------------
    # LOW RISK rules (stable / healthy signals)
    # -------------------------
    # Rule LOW-1: Positive month-over-month and low volatility
    if last_mom >= 0 and vol_pct < 30 and top10_share < 0.50:
        explanations.append("Rule LOW-1: Stable or growing recent sales, low volatility, and moderate customer concentration → Low risk.")
        return "LOW RISK – Business healthy; continue monitoring", explanations, metrics

    # Default fallback
    explanations.append("Fallback: No single rule hit strongly. Appears to be Low-to-Moderate risk — monitor key indicators.")
    return "LOW to MODERATE RISK – Monitor wellbeing of business", explanations, metrics


# -------------------------
# Example usage
# -------------------------
if __name__ == "__main__":
    # Load dataset
    df = load_data("/content/retail_sales_dataset.csv")

    # 1) Global assessment (whole business)
    label, reasons, diag = retail_risk_assessment(df)
    print("GLOBAL RISK LABEL:", label)
    print("REASONS:")
    for r in reasons:
        print(" -", r)
    print("\nKey diagnostics (snippet):")
    print(" Total revenue:", diag["total_revenue"])
    print(" Last month MoM %:", f"{diag['last_mom_pct']:.2f}%")
    print(" Top-1 customer share:", f"{diag['top1_customer_share']:.2%}")
    print(" Monthly volatility %:", f"{diag['monthly_sales_volatility_pct']:.2f}%")
    print(" Category shares:")
    print(diag["category_share"])

    # 2) Category-specific assessment example (e.g., 'Electronics')
    cat_label, cat_reasons, cat_diag = retail_risk_assessment(df, focus_category="Electronics")
    print("\nCATEGORY RISK LABEL (Electronics):", cat_label)
    for r in cat_reasons:
        print(" -", r)


GLOBAL RISK LABEL: CRITICAL RISK – Immediate attention required
REASONS:
 - Rule CRIT-1: Last month's sales dropped by more than 70% (large sudden collapse). Immediate investigation required.

Key diagnostics (snippet):
 Total revenue: 456000
 Last month MoM %: -96.58%
 Top-1 customer share: 0.44%
 Monthly volatility %: 36.19%
 Category shares:
{'Electronics': 0.34408991228070174, 'Clothing': 0.3411842105263158, 'Beauty': 0.31472587719298245}

CATEGORY RISK LABEL (Electronics): CRITICAL RISK – Immediate attention required
 - Rule CRIT-1: Last month's sales dropped by more than 70% (large sudden collapse). Immediate investigation required.


**Conclusion**

The Retail Sales Risk Screener successfully demonstrates how rule-based expert systems can analyze sales data and identify business risks with clear, explainable logic. By applying IF–THEN rules, the system detects declining sales, category weaknesses, customer dependency, and volatility. It provides meaningful insights that help retailers understand performance and take corrective action. Overall, the project shows that expert systems are an effective and transparent tool for retail decision support.


**Future Improvements**

Add machine learning models to predict future sales trends.

Integrate real-time dashboards for live monitoring.

Include inventory, promotions, and customer satisfaction data for deeper insights.

Allow the expert system to learn new rules automatically based on patterns.

Deploy the system as a web app for easier use by retail managers.